## Circuit Learning 

We start with importing libraries. We will use Xanadu's Open-Source platform PennyLane (https://pennylane.ai/) to perform this circuit learning task as it provides several in-built functions and libraries for direct use. 

In [19]:
import pennylane as qml
import numpy as np

Let's first talk about the problem, we need to create a circuit which gives |00> and |11> with equal probability. This will be a 2 qubit circuit with Bell state generation. Bell state is given by:

|Psi> = (|00> +/- |11>) / sqrt(2)

We have to create one of this state using only RX, RY and CNOT gates. Now, while it is strictly not necessary to have an intuition on how to create Bell states, it helps to have a more structured approach. |Psi>_plus_ can be created by a Hadamard operator on one qubit followed by CNOT gate on two qubits. 

Here we will use this knowledge as starting point and define the circuit as an RX and RY gate on qubit 0 followed by CNOT gate on qubit 0 and qubit 1 that will essentially qubit 1 to qubit 0. 

In [20]:
# Number of qubits in the circuit
number_qubits = 2

# Defining device. We will use Pennylane's default simulator which allows us to change number of measurements
dev = qml.device("default.qubit", wires=number_qubits, shots=1000, analytic=False)

# Circuit definition with two learning parameters
@qml.qnode(dev)
def circuit(parameters):
    qml.RX(parameters[0], wires=0)
    qml.RY(parameters[1], wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

Now we define a cost function that minimizes cost when there is equal probability of state |00> and |11>, this is when measurement of qubit 0 in Pauli Z is equal to 0 i.e. state in |0> and |1> with equal probability. 

In [7]:
def cost(parameters):
    p1, p2 = circuit(parameters)
    return np.absolute(p1)

We now optimize the parameters using Gradient Descent algorithm

In [8]:
def training(sampling):
    dev.shots = sampling
    initial_parameters = [0, 0] #We start with inital parameters 0, 0
    n_steps = 30 # Number of steps of optimization
    parameters_gd = initial_parameters.copy()
    # Here we use Adagrad gradient descent optimizer which adjusts learning rate based on past-gradient results
    opt = qml.AdagradOptimizer(stepsize=0.4)
    costs_gd = []
    for i in range(n_steps):
        costs_gd.append(cost(parameters_gd))
        parameters_gd = opt.step(cost, parameters_gd)

    print("------------------------")
    print('The cost evolution for {0} measurement sampling is: {1}'.format(sampling, costs_gd))
    print('\n \nThe final parameters are: {0}'.format(parameters_gd))
    qubit0, qubit1 = circuit(parameters_gd)
    print('\n \nThe qubit 0 and qubit 1 in Pauli-Z basis are {0} and {1}\n'.format(qubit0, qubit1))

Now we look at solutions of training function for different measurement samplings

In [9]:
# 1 Shot
training(1)

# 10 Shot
training(10)

# 100 Shot
training(100)

# 1000 Shot
training(1000)

------------------------
The cost evolution for 1 measurement sampling is: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

 
The final parameters are: [0.047328272985473346, -1.2329112386331036]

 
The qubit 0 and qubit 1 in Pauli-Z basis are -1.0 and -1.0

------------------------
The cost evolution for 10 measurement sampling is: [1.0, 0.6, 0.8, 0.6, 0.4, 0.6, 0.2, 0.6, 0.4, 0.6, 0.4, 0.2, 0.8, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.4, 0.2, 0.2, 0.0, 0.6, 0.0, 0.4, 0.4, 0.0, 0.0, 0.0]

 
The final parameters are: [-1.6846767983468014, 1.9074549451934852]

 
The qubit 0 and qubit 1 in Pauli-Z basis are 0.2 and 0.2

------------------------
The cost evolution for 100 measurement sampling is: [1.0, 0.7, 0.62, 0.16, 0.2, 0.04, 0.02, 0.02, 0.12, 0.18, 0.08, 0.0, 0.04, 0.1, 0.18, 0.04, 0.16, 0.1, 0.14, 0.08, 0.02, 0.04, 0.14, 0.12, 0.04, 0.04, 0.04, 0.04, 0.0, 0.22]

 
The final parameters are:

Now let's look at results for different number of measurement sampling. As expected, with only 1 measurement, cost function is always 1 as expectation value is always 1 or -1. As we increase number of measurements, cost function starts to minimize and gets closer to 0. With 1000 measurements, it is almost perfectly in Bell state. This is verfied in two-fold way, one is that expectation value of qubit 0 is 0 so it is equally in |0> and |1> state, while qubit 1 also always have same expectation value as qubit 0. However, right now we can't tell if parameters are for state |Psi>_plus_ or |Psi>_minus_

#### Bonus
How do we ensure that resulting state after circuit is trained is |Psi>_plus_ = (|00> + |11>) / sqrt(2)?

Instead of measuring the circuit in Pauli-Z basis, we have to measure it in Pauli-X basis on both qubits. In PennyLane, it is possible to do such multi-qubit observation in Pauli-X basis with tensor observables. We modify the circuit and cost function ( -1 multiplier in cost function as we want |Psi>_plus_ which has expectation value of 1 in Pauli-X basis) to to ensure generation of a specific Bell state

In [40]:
# Modified circuit
@qml.qnode(dev)
def circuit_psi(parameters):
    qml.RX(parameters[0], wires=0)
    qml.RY(parameters[1], wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliX(0) @ qml.PauliX(1))

# Modified cost function
def cost2(parameters):
    p = circuit_psi(parameters)
    return -1*p 

In [41]:
dev.shots = 100
initial_parameters = [0.14, 0.14] #We start with inital parameters 0, 0
n_steps = 100 # Number of steps of optimization
parameters_gd = initial_parameters.copy()
# Here we also use Adagrad gradient descent optimizer
opt = qml.AdagradOptimizer(stepsize=0.3)
costs_gd = []
for i in range(n_steps):
    costs_gd.append(cost2(parameters_gd))
    parameters_gd = opt.step(cost2, parameters_gd)

print("------------------------")
print('The cost evolution for {0} measurement sampling is: {1}'.format(dev.shots, costs_gd))
print('\n \nThe final parameters are: {0}'.format(parameters_gd))
psi_state = circuit_psi(parameters_gd)
print('\n \nThe Psi state in X-basis has expectation value: {0}\n'.format(psi_state))

------------------------
The cost evolution for 100 measurement sampling is: [-0.28, -0.46, -0.62, -0.66, -0.68, -0.84, -0.9, -0.96, -0.92, -0.98, -0.94, -0.92, -0.98, -0.98, -1.0, -0.96, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -0.98, -1.0, -1.0, -0.98, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -0.98, -1.0, -1.0, -0.98, -1.0, -0.98, -1.0, -1.0, -1.0, -0.98, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -0.98, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -0.98, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]

 
The final parameters are: [0.021035050539376897, 1.55979636262034]

 
The Psi state in X-basis has expectation value: 1.0



The circuit learnt using gradient descent is: 

Rx (0) on Q0 -- Ry(pi/2) on Q0 -- CNOT on (QO, Q1)